In [1]:
%load_ext autoreload
%autoreload 2

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import EVO
from pylabrobot.resources.tecan.tecan_decks import TecanDeck, EVO200_NUM_RAILS, EVO200_SIZE_X, EVO200_SIZE_Y, EVO200_SIZE_Z

from load_decks import build_layout

deck = build_layout()

backend = EVO()



lh = LiquidHandler(backend=backend, deck=deck)


In [2]:
await lh.setup()

In [6]:
from pylabrobot.resources import Coordinate, Deck, Tip, Liquid, TecanTip400


lh.clear_head_state()
lh.update_head_state({0:TecanTip400()})
lh.update_head_state({1:TecanTip400()})
lh.update_head_state({2:TecanTip400()})
lh.update_head_state({3:TecanTip400()})
lh.update_head_state({4:TecanTip400()})
lh.update_head_state({5:TecanTip400()})
lh.update_head_state({6:TecanTip400()})
lh.update_head_state({7:TecanTip400()})



In [5]:
lh.deck.serialize()

{'name': 'deck',
 'type': 'TecanDeck',
 'size_x': 1915,
 'size_y': 780,
 'size_z': 765,
 'location': {'x': 0, 'y': 0, 'z': 0, 'type': 'Coordinate'},
 'category': 'deck',
 'children': [{'name': 'plate_carrier_1',
   'type': 'TecanPlateCarrier',
   'size_x': 149.0,
   'size_y': 295.0,
   'size_z': 6.0,
   'roma_x': 2056,
   'roma_y': 441,
   'roma_z_safe': 610,
   'roma_z_travel': 2418,
   'roma_z_end': 2503,
   'location': {'x': 238.0, 'y': 61.0, 'z': 0, 'type': 'Coordinate'},
   'category': 'tecan_plate_carrier',
   'model': 'MP_3Pos_Flat',
   'off_x': 12.0,
   'off_y': 11.0,
   'children': [{'spot': 0,
     'name': 'carrier-plate_carrier_1-spot-0',
     'type': 'CarrierSite',
     'size_x': 127.0,
     'size_y': 85.5,
     'size_z': 0,
     'location': {'x': 10.4, 'y': 11.5, 'z': 10.0, 'type': 'Coordinate'},
     'category': 'carrier_site',
     'model': None,
     'children': [{'name': 'reaction_setup',
       'type': 'TecanPlate',
       'size_x': 127.8,
       'size_y': 87.4,
     

In [88]:
deck = build_layout()

lh.deck = deck
# lh.deck.assign_child_resource(reaction_96, resource_to)

In [19]:
from pylabrobot.liquid_handling.standard import PipettingOp, Pickup, Aspiration, Dispense, Drop, Move
from pylabrobot.resources import Coordinate, Deck, Tip, Liquid, TecanTip400
from pylabrobot.serializer import serialize, deserialize
from typing import Any, Coroutine, List, Tuple, Type, TypeVar, Optional, cast

class ErrorResponse(Exception):
  def __init__(self, data: dict, status_code: int):
    self.data = data
    self.status_code = status_code



data = {
  "channels":[
    {"blow_out_air_volume":0,
     "flow_rate":None,
     "liquid":None,
     "liquid_height":None,
     "offset":{"x":0,"y":0,"z":0},
     "resource_name":"solvent_trough_C3",
     "tip":{"fitting_depth":37,"has_filter":False,"maximal_volume":400,"tip_type":"Standard","total_tip_length":154,"type":"TecanTip"},"volume":350}
  ],
  "use_channels":[0]
}
aspirations = []
for sc in data["channels"]:
  try:
    resource = lh.deck.get_resource(sc["resource_name"])
  except ValueError as exc:
    raise ErrorResponse({"error": f"resource with name '{sc['resource_name']}' not found"},
                        404) from exc
  if not "tip" in sc:
    raise ErrorResponse({"error": "missing key in json data: tip"}, 400)
  tip = cast(Tip, deserialize(sc["tip"]))
  if not "offset" in sc:
    raise ErrorResponse({"error": "missing key in json data: offset"}, 400)
  offset = cast(Coordinate, deserialize(sc["offset"]))
  volume = sc["volume"]
  flow_rate = sc["flow_rate"]
  liquid_height = sc["liquid_height"]
  blow_out_air_volume = sc["blow_out_air_volume"]
  liquid = cast(Liquid, deserialize(sc["liquid"]))
  aspirations.append(Aspiration(resource=resource, tip=tip, offset=offset, volume=volume,
    flow_rate=flow_rate, liquid_height=liquid_height, blow_out_air_volume=blow_out_air_volume,
    liquid=liquid))
use_channels = data["use_channels"]

await lh.aspirate(
  resources=[a.resource for a in aspirations],
  vols=[a.volume for a in aspirations],
  offsets=[a.offset for a in aspirations],
  flow_rates=[a.flow_rate for a in aspirations],
  use_channels=use_channels
)

TecanError: ('Subdevice error', 'C5', 28)

In [31]:
deck = build_layout()

lh.deck = deck


In [33]:
print(GripDirection.BACK.value)

2


In [32]:
from pylabrobot.liquid_handling.standard import (
  GripDirection
)





resource_name = "reaction_setup"
to = "carrier-plate_carrier_1-spot-3"
resource = deck.get_resource(resource_name)
resource_to = deck.get_resource(to)


# current_app.lh.deck.unassign_child_resource(resource)
# current_app.lh.deck.assign_child_resource(resource, resource_to)

# move = Move(resource=resource, to=to)

# print(resource_name, resource_to.get_absolute_location())
await lh.move_plate(resource, resource_to, put_direction=GripDirection.BACK)



In [92]:
resource_name = "reaction_setup"
to = "carrier-plate_carrier_1-spot-0"
resource = deck.get_resource(resource_name)
resource_to = deck.get_resource(to)


# current_app.lh.deck.unassign_child_resource(resource)
# current_app.lh.deck.assign_child_resource(resource, resource_to)

# move = Move(resource=resource, to=to)

# print(resource_name, resource_to.get_absolute_location())
await lh.move_plate(resource, resource_to)


Move(resource=TecanPlate(name=reaction_setup, size_x=127.8, size_y=87.4, size_z=7.6, location=(000.000, 000.000, 000.000), one_dot_max=0), to=Coordinate(x=248.4, y=72.5, z=10.0), intermediate_locations=[], resource_offset=Coordinate(x=0, y=0, z=0), to_offset=Coordinate(x=0, y=0, z=0), pickup_distance_from_top=13.2, get_direction=<GripDirection.FRONT: 1>, put_direction=<GripDirection.FRONT: 1>)
debug roma 3
debug roma 4
debug roma 5
debug roma 1
debug roma 3
debug roma 4
debug roma 5
debug roma 2
C5 45 10540 10540
debug roma movement 1
C5 45 10540 10540
C5 45 10540 10540
debug roma movement 2
debug roma movement 3.1
C5 45 10540 10540
debug roma movement 3.1.1
C5 45 10540 10540
debug roma movement 3.1.2
C5 45 10540 10540
debug roma movement 3.1.3
C5 45 10540 3540
debug roma movement 3.1.4
C5 45 10540 3540
debug roma movement 3.1.5
C5 45 10540 3540
debug roma movement 3.2
debug roma movement 3
C5 45 3540 3540
C5 45 3540 3540
C5 45 3540 3540
debug roma movement 4
reassign0
reassign0.5
reas